# Build FakeHuayi backend

## Basic Use

Required files:

- ./huayi_providers/fake_huayi/
    - \_\_init\_\_.py
    - fake_huayi.py
    - props_huayi.json
    - conf_huayi.json
    - defs_huayi.json (if Pulse Backend is applied, TODO)
- ./
    - qubits_data.csv
    - gates_data.csv

Import FakeHuayi backend

```
from huayi_providers.fake_huayi import *
FackHuayi()    # for V1 backend
FackHuayiV2()  # for V2 backend
```

Generate noise model
```
from qiskit_aer.noise.noise_model import NoiseModel
noise_Huayi = NoiseModel.from_backend(FakeHuayi())
```

Create .json files that required to build the backend

```
from huayi_providers.backend_build import build_from_file
c = build_from_file(backend_name="huayi",
                    backend_version="x.x.x",
                    qubits_data="qubits_data.csv",
                    gates_data="gates_data.csv")
```

## File Structure

#### Expriment data

**qubits_data.csv** contains the information of qubits, including
- T1 time (ms)
- T2 time (ms)
- frequency (MHz)
- readout error rate
- Probability of finding 0 when prepared in 1
- Probability of finding 1 when prepared in 0
- readout length (us)

All information should be accompanied with the measurement date and time.

**gates_data.csv** contains the information of gates, including
- qubits
- gate type
- error rate
- length (us)
- gate name (optional)

The gate error and length are measured from experiment, and should be accompanied with the measurement date and time.

#### Dictionaries of the backend properties and configurations

**props_huayi.json**
```
{'backend_name': 'fakehuayi',
 'backend_version='0.0.1',
 'last_update_date': now_time(),
 'qubits': [q1, q2, ...],
 'gates': [g1, g2, ...],
 'general': []}
```
 qubit info (q):
```
 [{'date': ['T1_date'], 'name': 'T1', 'unit': 'ms', 'value': ['T1']},
  {'date': ['T2_date'], 'name': 'T2', 'unit': 'ms', 'value': ['T2']},
  {'date': ['frequency_date'], 'name': 'frequency', 'unit': 'MHz', 'value': ['frequency']},
  {'date': ['readout_error_date'], 'name': 'readout_error', 'unit': '', 'value': ['readout_error']},
  {'date': ['prob_meas0_prep1_date'], 'name': 'prob_meas0_prep1', 'unit': '', 'value': ['prob_meas0_prep1']},
  {'date': ['prob_meas1_prep0_date'], 'name': 'prob_meas1_prep0', 'unit': '', 'value': ['prob_meas1_prep0']},
  {'date': ['readout_length_date'], 'name': 'readout_length', 'unit': 'us', 'value': ['readout_length']}]
```
gate info (g):
```
{'qubits': ['qubits'],
 'gate': ['gate'],
 'parameters': [{'date': ['error_date'],
                 'name': 'gate_error',
                 'unit': '',
                 'value': ['gate_error']},
                {'date': ['length_date'],
                 'name': 'gate_length',
                 'unit': 'ms',
                 'value': ['gate_length']}],
 'name': ['name']}
```

**conf_huayi.json**

#### Dictionary of the Noise Model

The keys in NoiseModel are
- basis gates ({'id', 'x', 'sx', 'rz', 'cx'})
- noise instructions ({'id', 'x', 'sx', 'cx', 'measure', 'reset', ''})
- noise qubits ({0, ... , 26})
- default quantum errors (set None by default)
- default readout errors (set None by default)
- local quantum errors
- local readout errors
- custom noise passes (set None by default)

**local readout errors**

```
noise_model._local_readout_errors = {(0,):ReadoutError, 
                                     (1,):ReadoutError, 
                                     (2,):ReadoutError, ...}
```

``ReadoutError`` is essentially a matrix, the off-diagonal terms corresponds to ``prob_meas0_prep1`` and ``prob_meas1_prep0``

If ``prob_meas0_prep1`` and ``prob_meas1_prep0`` are given, ``readout_error`` will be ignored.


**local quantum errors**

The errors of ``gate`` applied to indices ``ind = (i,) for one-bit gate, (i,j) for two-bit gate`` are stored in

```
noise_model._local_quantum_errors[gate][ind] = {
  '_id': xxxx,
  '_probs': list_of_probs,
  '_circs': list_of_circs,
  '_qargs': None,
  '_op_shape': OpShape(num_qargs_l, num_qargs_r)}
```

``_circs`` includes all possible matrices with the same shape of the corresponding gate

``_probs`` is the corresponding probability of each circ

For one-bit gate, ``_circ`` includes `I`, `X`, `Y`, `Z`. The probs of nonidentity matrices are ``gate_error/2``. ``gate_error`` in `.csv` file refers to the measurement of `1-(I+Z)`.

For two-bit gate, ``_circ`` includes `II`, `IX`, `IY`, `IZ`, `XI`, `XX`, `XY`, `XZ`, `YI`, `YX`, `YY`, `YZ`, `ZI`, `ZX`, `ZY`, `ZZ`. The probs of nonidentity matrices are ``gate_error/12``. ``gate_error`` in `.csv` file refers to the measurement of `1-(II+IZ+ZI+ZZ)`.



## Build a backend from csv files

In [5]:
# Generate props_huayi.json from .csv experimental data
from huayi_providers.backend_build import build_from_file

"""
'gates_radius' defines a coupling_map with the distance between two-qubit gates smaller (or equal) to gates_radius
But suggest not to use 'gates_radius', but define the 'coupling_map' in transpile()

use the following map for finite-range coupling:

       coupling_map = [[i,j] for i in range(n_qubits)
                             for j in list(range(max(0,i-gates_radius),i))+
                                      list(range(i+1,min(i+1+gates_radius,n_qubits)))]
"""
c = build_from_file(backend_name='huayi12',
       backend_version='0.0.2',
       qubits_data='qubits_data_12.csv', 
       gates_data='gates_data_12.csv',)


Successfully created props_huayi12.json
Successfully created conf_huayi12.json
New backends created, please import the backends with:
from huayi_providers.fake_huayi12 import FakeHuayi12, FakeHuayi12V2


## Check if the new backend works

In [63]:
from huayi_providers.fake_huayi8 import FakeHuayi8, FakeHuayi8V2
from qiskit.circuit.library import EfficientSU2
from qiskit import transpile
from qiskit import QuantumCircuit

n_qubits = 4
ansatz = EfficientSU2(n_qubits, 
                      su2_gates=['ry'], 
                      entanglement="linear", 
                      reps=1,
                      flatten=True)
print("Efficient SU2 ansatz")
print(ansatz.draw(fold=140, idle_wires=False))


coupling_map = [[i,j] for i in range(8)
                      for j in list(range(max(0,i-3),i))+
                               list(range(i+1,min(i+1+3,8)))]
c_huayi = transpile(ansatz, backend=FakeHuayi8(), optimization_level=3)
print("Transpileed with Huayi backend, depth = {}".format(c_huayi.depth()))
print(c_huayi.draw(fold=140, idle_wires=False))


Efficient SU2 ansatz
     ┌──────────┐     ┌──────────┐                        
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[4]) ├────────────────────────
     ├──────────┤┌─┴─┐└──────────┘┌──────────┐            
q_1: ┤ Ry(θ[1]) ├┤ X ├─────■──────┤ Ry(θ[5]) ├────────────
     ├──────────┤└───┘   ┌─┴─┐    └──────────┘┌──────────┐
q_2: ┤ Ry(θ[2]) ├────────┤ X ├─────────■──────┤ Ry(θ[6]) ├
     ├──────────┤        └───┘       ┌─┴─┐    ├──────────┤
q_3: ┤ Ry(θ[3]) ├────────────────────┤ X ├────┤ Ry(θ[7]) ├
     └──────────┘                    └───┘    └──────────┘
Transpileed with Huayi backend, depth = 13
global phase: π
         ┌──────────┐┌─────────┐┌───────┐   ┌─────────┐ ┌───────┐   ┌──────────┐                                             
q_1 -> 2 ┤ Ry(θ[1]) ├┤ Ry(π/2) ├┤ Rx(π) ├─■─┤ Ry(π/2) ├─┤ Rx(π) ├─■─┤ Ry(θ[5]) ├─────────────────────────────────────────────
         ├──────────┤├─────────┤├───────┤ │ └─────────┘ └───────┘ │ └──────────┘            ┌─────────┐ ┌───────┐┌──────────┐
q_3 -> 3 ┤ 

I have manually forced the CZ gate error very large (~0.8) except for the couplings between [1,3], [1,5] and [3,7]
The transpiled circuit indeed only connects the low-error qubits

## Create random properties of qubits and gates

In [123]:
import pandas as pd
import random
from huayi_providers.backend_build import now_time

n_qubits = 12

qubits_info = pd.DataFrame(columns=['T1','T1_date', 
                                    'T2','T2_date', 
                                    'frequency', 'frequency_date', 
                                    'readout_error', 'readout_error_date', 
                                    'prob_meas0_prep1','prob_meas0_prep1_date', 
                                    'prob_meas1_prep0', 'prob_meas1_prep0_date', 
                                    'readout_length', 'readout_length_date'])
for i in range(n_qubits):
    qubits_info.loc[i] = [abs(random.gauss(50e3, 1e3)), now_time(), # T1
                         abs(random.gauss(1000, 50)), now_time(),   # T2
                         abs(random.gauss(1, 0.05)), now_time(),    # frequency
                         abs(random.gauss(0, 0.2e-2)), now_time(), # readout_error
                         abs(random.gauss(0, 0.1e-2)), now_time(), # prob_0_1
                         abs(random.gauss(0, 0.1e-2)), now_time(), # prob_1_0
                         abs(random.gauss(0.1, 0.01)), now_time(),] # readout_length
with open('qubits_data_{}.csv'.format(n_qubits), 'w') as f:
    qubits_info.to_csv(f)


gates_info = pd.DataFrame(columns=['qubits',
                                   'gate',
                                   'gate_error', 'error_date',
                                   'gate_length', 'length_date',
                                   'name'])
iter = -1
for gate in ['id', 'rx' ,'ry']:
    for j in range(n_qubits):
        iter += 1
        gates_info.loc[iter] = ['['+str(j)+']',
                                gate,
                                abs(random.gauss(0, 0.003e-2)), now_time(), # 1qb gate error
                                random.gauss(150, 10), now_time(),
                                gate+str(j)]
for gate in ['cz']:
    for i in range(n_qubits):
        for j in list(range(i))+list(range(i+1,n_qubits)):
            iter += 1
            gates_info.loc[iter] = ['['+str(i)+','+str(j)+']',
                                    gate,
                                    abs(random.gauss(0, 0.2e-2)), now_time(), # 2qb gate error
                                    abs(random.gauss(150, 10)), now_time(),
                                    gate+str(i)+'_'+str(j)]
    

with open('qubits_data_{}.csv'.format(n_qubits), 'w') as f:
    qubits_info.to_csv(f, index=False, lineterminator='\n')
with open('gates_data_{}.csv'.format(n_qubits), 'w') as f:
    gates_info.to_csv(f, index=False, lineterminator='\n')


display(pd.read_csv('qubits_data_{}.csv'.format(n_qubits)))
display(pd.read_csv('gates_data_{}.csv'.format(n_qubits)))

,T1,T1_date,T2,T2_date,frequency,frequency_date,readout_error,readout_error_date,prob_meas0_prep1,prob_meas0_prep1_date,prob_meas1_prep0,prob_meas1_prep0_date,readout_length,readout_length_date
0,51337.721470,2023-12-24T13:57+08:00,919.578212,2023-12-24T13:57+08:00,1.043731,2023-12-24T13:57+08:00,0.003890,2023-12-24T13:57+08:00,0.000518,2023-12-24T13:57+08:00,0.000052,2023-12-24T13:57+08:00,0.110796,2023-12-24T13:57+08:00
1,48984.200574,2023-12-24T13:57+08:00,973.688318,2023-12-24T13:57+08:00,1.129420,2023-12-24T13:57+08:00,0.002230,2023-12-24T13:57+08:00,0.000703,2023-12-24T13:57+08:00,0.000177,2023-12-24T13:57+08:00,0.096785,2023-12-24T13:57+08:00
2,51095.331553,2023-12-24T13:57+08:00,1049.568846,2023-12-24T13:57+08:00,1.039395,2023-12-24T13:57+08:00,0.000854,2023-12-24T13:57+08:00,0.001723,2023-12-24T13:57+08:00,0.001435,2023-12-24T13:57+08:00,0.107142,2023-12-24T13:57+08:00
3,50716.072564,2023-12-24T13:57+08:00,952.152530,2023-12-24T13:57+08:00,0.942015,2023-12-24T13:57+08:00,0.001402,2023-12-24T13:57+08:00,0.001195,2023-12-24T13:57+08:00,0.002070,2023-12-24T13:57+08:00,0.091311,2023-12-24T13:57+08:00
4,48946.302261,2023-12-24T13:57+08:00,1002.176274,2023-12-24T13:57+08:00,0.935092,2023-12-24T13:57+08:00,0.002581,2023-12-24T13:57+08:00,0.000388,2023-12-24T13:57+08:00,0.001841,2023-12-24T13:57+08:00,0.079607,2023-12-24T13:57+08:00
5,51670.203438,2023-12-24T13:57+08:00,1011.826663,2023-12-24T13:57+08:00,0.959945,2023-12-24T13:57+08:00,0.002640,2023-12-24T13:57+08:00,0.000215,2023-12-24T13:57+08:00,0.000085,2023-12-24T13:57+08:00,0.097239,2023-12-24T13:57+08:00
6,49566.233242,2023-12-24T13:57+08:00,1023.205074,2023-12-24T13:57+08:00,1.041527,2023-12-24T13:57+08:00,0.001742,2023-12-24T13:57+08:00,0.000211,2023-12-24T13:57+08:00,0.001324,2023-12-24T13:57+08:00,0.102663,2023-12-24T13:57+08:00
7,49853.996938,2023-12-24T13:57+08:00,1012.747696,2023-12-24T13:57+08:00,0.980903,2023-12-24T13:57+08:00,0.003264,2023-12-24T13:57+08:00,0.001910,2023-12-24T13:57+08:00,0.000438,2023-12-24T13:57+08:00,0.113534,2023-12-24T13:57+08:00
8,50183.857713,2023-12-24T13:57+08:00,1007.157378,2023-12-24T13:57+08:00,1.022920,2023-12-24T13:57+08:00,0.002200,2023-12-24T13:57+08:00,0.000611,2023-12-24T13:57+08:00,0.000033,2023-12-24T13:57+08:00,0.102306,2023-12-24T13:57+08:00
9,50477.823699,2023-12-24T13:57+08:00,1052.898087,2023-12-24T13:57+08:00,0.953032,2023-12-24T13:57+08:00,0.003403,2023-12-24T13:57+08:00,0.001030,2023-12-24T13:57+08:00,0.000539,2023-12-24T13:57+08:00,0.100328,2023-12-24T13:57+08:00


,qubits,gate,gate_error,error_date,gate_length,length_date,name
0,[0],id,0.000026,2023-12-24T13:57+08:00,157.490463,2023-12-24T13:57+08:00,id0
1,[1],id,0.000008,2023-12-24T13:57+08:00,154.839036,2023-12-24T13:57+08:00,id1
2,[2],id,0.000018,2023-12-24T13:57+08:00,146.050516,2023-12-24T13:57+08:00,id2
3,[3],id,0.000044,2023-12-24T13:57+08:00,138.985736,2023-12-24T13:57+08:00,id3
4,[4],id,0.000041,2023-12-24T13:57+08:00,136.901056,2023-12-24T13:57+08:00,id4
...,...,...,...,...,...,...,...
163,"[11,6]",cz,0.001038,2023-12-24T13:57+08:00,143.855615,2023-12-24T13:57+08:00,cz11_6
164,"[11,7]",cz,0.000376,2023-12-24T13:57+08:00,161.486479,2023-12-24T13:57+08:00,cz11_7
165,"[11,8]",cz,0.002952,2023-12-24T13:57+08:00,163.632383,2023-12-24T13:57+08:00,cz11_8
166,"[11,9]",cz,0.001095,2023-12-24T13:57+08:00,154.967007,2023-12-24T13:57+08:00,cz11_9
